In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Load data for the first app
debt = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/Ratio_of_Education_Debt.csv', encoding='utf-8')
birthrate = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/자치구별_출산율f.csv', encoding='utf-8')
merged_df1 = pd.merge(debt, birthrate, on='자치구별', how='left')

# Load data for the second app
df_2015 = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/2015 last.csv', encoding='euc-kr')
df_2016 = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/2016 last.csv', encoding='euc-kr')
df_2017 = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/2017last.csv', encoding='euc-kr')

# Load data for the third app
finance = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/Educational_Financial_Support.csv', encoding='utf-8')
birthrate_2 = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/자치구별_출산율f.csv', encoding='utf-8')
merged_df = pd.merge(finance, birthrate_2, on='자치구별', how='left')

# Read data for the fourth app
education_expense = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/Education_Expense.csv', encoding='euc-kr')
birthrate_combined = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/자치구별_출산율f.csv', encoding='utf-8')
merged_df_combined = pd.merge(education_expense, birthrate_combined, on='자치구별', how='left')

# Add the new code for the Fifty-First Tab with the pie chart
df_pie = pd.DataFrame()

finance_pie = pd.read_csv('/Users/hyeongseokkim/Desktop/데이터마이닝 자료/babyboom/data/1인가구%.csv', encoding='utf-8')

fin_pie = finance_pie.loc[:, ['동별', '1인가구%']].groupby(by='동별', as_index=False).sum()

trace_pie = go.Pie(
    labels=fin_pie['동별'],
    values=fin_pie['1인가구%']
)

data_pie = [trace_pie]
layout_pie = go.Layout(title='1인가구 비율')

fig_pie = go.Figure(data_pie, layout_pie)

# Create Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='What are the factors that influence the birth rate?'),

    dcc.Tabs([
        # Second Tab
        dcc.Tab(label='Real Estate Dashboard', children=[
            html.Div([
                html.H2("Seoul Real Estate Dashboard"),

                # Dropdown for selecting the year
                dcc.Dropdown(
                    id='year-dropdown2',
                    options=[
                        {'label': '2015', 'value': '2015'},
                        {'label': '2016', 'value': '2016'},
                        {'label': '2017', 'value': '2017'}
                    ],
                    value='2015',
                    style={'width': '50%'}
                ),

                # Graph component for displaying plots
                dcc.Graph(id='dashboard-graph')
            ])
        ]),

        # First Tab
        dcc.Tab(label='Birthrate and Education Debt Ratio', children=[
            html.Div([
                html.H2("Birthrate and Education Debt Ratio Dashboard"),
                dcc.Dropdown(
                    id='year-dropdown1',
                    options=[{'label': str(year), 'value': str(year)} for year in range(2007, 2015)],
                    value='2007',
                    style={'width': '50%'}
                ),
                dcc.Graph(id='subplot-graph')
            ]),
        ]),

        # Third Tab
        dcc.Tab(label='Educational Finance Dashboard', children=[
            html.Div([
                html.H2("Educational Finance Dashboard"),

                dcc.Dropdown(
                    id='year-dropdown-education',
                    options=[
                        {'label': f'Year {year}', 'value': year} for year in range(2007, 2015)
                    ],
                    value='2007',
                    style={'width': '50%'}
                ),
                dcc.Graph(id='education-dashboard-plot')
            ])
        ]),

        # Fourth Tab (Integrated into the Third Tab)
        dcc.Tab(label='Education Expense and Birthrate Analysis', children=[
            html.Div([
                html.H2("Education Expense and Birthrate Analysis"),

                html.Label("Select Year:"),
                dcc.Dropdown(
                    id='year-dropdown-fourth',
                    options=[
                        {'label': '2019', 'value': '2019'},
                        {'label': '2020', 'value': '2020'},
                        {'label': '2021', 'value': '2021'},
                        {'label': '2022', 'value': '2022'},
                    ],
                    value='2019'
                ),

                dcc.Graph(id='combined-plot-fourth')
            ])
        ]),
        # Fifty-First Tab (Pie Chart)
        dcc.Tab(label='1인가구 Pie Chart', children=[
            html.Div([
                html.H2("1인가구 비율 Pie Chart"),
                dcc.Graph(figure=fig_pie)
            ]),
        ]),

    ]),
])

# Callbacks for the first app
@app.callback(
    Output('subplot-graph', 'figure'),
    [Input('year-dropdown1', 'value')]
)
def update_graph1(selected_year):
    bar_trace = go.Bar(x=merged_df1['자치구별'], y=merged_df1[f'{selected_year}_y'], name='출산율')
    line_trace = go.Scatter(x=merged_df1['자치구별'], y=merged_df1[f'{selected_year}_x'], mode='lines', name='부채 중 교육비 비율')

    fig = make_subplots(rows=1, cols=1, specs=[[{'secondary_y': True}]])
    fig.add_trace(bar_trace, secondary_y=False)
    fig.add_trace(line_trace, secondary_y=True)

    fig.update_layout(title_text=f'{selected_year}년 출산율과 부채 중 교육비 비율')
    fig.update_xaxes(title_text='자치구별')
    fig.update_yaxes(title_text='출산율', secondary_y=False)
    fig.update_yaxes(title_text='부채 중 교육비 비율', secondary_y=True)

    return fig


# Callbacks for the second app
@app.callback(
    Output('dashboard-graph', 'figure'),
    [Input('year-dropdown2', 'value')]
)
def update_graph2(selected_year):
    # Select DataFrame based on the selected year
    if selected_year == '2015':
        df = df_2015
    elif selected_year == '2016':
        df = df_2016
    elif selected_year == '2017':
        df = df_2017

    # Create subplot
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    fig.add_trace(go.Scatter(x=df['시군구'], y=df['출산율'], mode='lines', name='출산율'), secondary_y=True)
    fig.add_trace(go.Bar(x=df['시군구'], y=df['평당가격'], name='부동산 가격'), secondary_y=False)

    # Update layout
    fig.update_layout(
        title_text=f'{selected_year} 서울시 시군구 부동산 가격',
        xaxis_title='시군구',
        yaxis_title='Y1 Axis',
        height=400
    )
    fig.update_yaxes(title_text='부동산 가격', secondary_y=False)
    fig.update_yaxes(title_text='출산율', secondary_y=True)

    return fig

# Callbacks for the third app (Updated with the Fourth Tab code)
@app.callback(
    Output('education-dashboard-plot', 'figure'),
    [Input('year-dropdown-education', 'value')]
)
def update_education_dashboard(selected_year):
    fig = make_subplots(rows=2, cols=1, specs=[[{'secondary_y': True}], [{'secondary_y': True}]])

    bar_trace = go.Bar(x=merged_df['자치구별'], y=merged_df[f'{selected_year}_y'], name=f'출산율 {selected_year}')
    line_trace = go.Scatter(x=merged_df['자치구별'], y=merged_df[f'{selected_year}_x'], mode='lines', name=f'교육재정지원현황 {selected_year}')

    fig.add_trace(bar_trace, row=1, col=1, secondary_y=False)
    fig.add_trace(line_trace, row=1, col=1, secondary_y=True)

    fig.update_layout(title_text=f'{selected_year}년 출산율과 교육재정지원현황 대시보드', height=800, width=1000)
    fig.update_xaxes(title_text='자치구별', row=2, col=1)
    fig.update_yaxes(title_text='출산율', secondary_y=False, row=1, col=1)
    fig.update_yaxes(title_text='교육재정지원현황', secondary_y=True, row=1, col=1)

    return fig


# Callbacks for the additional dropdown and graph in the fourth tab
@app.callback(
    Output('combined-plot-fourth', 'figure'),
    [Input('year-dropdown-fourth', 'value')]
)
def update_combined_plot_fourth(selected_year):
    filtered_df = merged_df_combined[merged_df_combined[f'{selected_year}_x'].notna() & merged_df_combined[f'{selected_year}_y'].notna()]

    fig = make_subplots(rows=1, cols=1, specs=[[{'secondary_y': True}]])

    bar_trace = go.Bar(x=filtered_df['자치구별'], y=filtered_df[f'{selected_year}_y'], name='출산율')
    line_trace = go.Scatter(x=filtered_df['자치구별'], y=filtered_df[f'{selected_year}_x'], mode='lines', name='교육 관련 지출 비용')

    fig.add_trace(bar_trace, secondary_y=False)
    fig.add_trace(line_trace, secondary_y=True)

    fig.update_layout(title_text=f'{selected_year}년 출산율과 교육 관련 지출 비용')
    fig.update_xaxes(title_text='자치구별')
    fig.update_yaxes(title_text='출산율', secondary_y=False)
    fig.update_yaxes(title_text='교육 관련 지출 비용', secondary_y=True)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


/var/folders/8m/0ks16rl51l1_8514jzz3_f280000gn/T/ipykernel_1558/2211079907.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/8m/0ks16rl51l1_8514jzz3_f280000gn/T/ipykernel_1558/2211079907.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2023 13:26:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2023 13:26:46] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2023 13:26:46] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2023 13:26:46] "GET /_dash-component-suites/plotly/package_data/plotly.min.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2023 13